# Homework 4. Suggest

Artem Chernitsa, a.chernitsa@innopolis.university, B20-AI-01  

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) data structure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on a trie;
3. Measure suggestion speed;
4. [M] Add spellcheck to suggest.


![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [1]:
!pip install pygtrie pyspellchecker numpy pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.0 MB/s eta 0:00:00


In [2]:
import os
import math
import gzip
import time
import shutil
import itertools

from collections import Counter

import pygtrie
import requests
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm import tqdm
from pprint import pprint
from matplotlib import pyplot as plt
from spellchecker import SpellChecker

## 1. Build a trie upon a dataset

### 1.1. [5] Read the dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol)(we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset. Pass the assert.

In [3]:
def download_dataset(url, filename="aol.txt", data_dir="./data"): 
    try: 
        os.makedirs(data_dir, exist_ok=True)
        archivename = f"{filename}.gz"
        path = os.path.join(data_dir, archivename)
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)

        with gzip.open(path, mode="rb") as f_in, open(os.path.join(data_dir, filename), mode="wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

            return True
    except requests.exceptions.RequestException as e:
        print(f"Something went wrong: {e}")
        return False

url = "https://github.com/IUCVLab/information-retrieval/blob/main/datasets/aol/user-ct-test-collection-01.txt.gz?raw=true"
download_dataset(url)

True

In [4]:
def read_dataset(filename="aol.txt", data_dir="./data"):
    path = os.path.join(data_dir, filename)
    df = pd.read_csv(path, delimiter = "\t")
    return df

In [5]:
aol_data = read_dataset()

assert aol_data.shape[0] == 3558411, "Dataset size does not match"

In [6]:
aol_data.sample(10)

,AnonID,Query,QueryTime,ItemRank,ClickURL
2524673,9051983,recent,2006-05-28 00:50:04,NaN,NaN
848204,2157071,www.myspace,2006-05-13 16:12:29,NaN,NaN
144622,352728,poseys,2006-05-13 10:40:53,4.0,http://www.findaflorist.com
2248787,7271448,nurses salary,2006-05-08 22:04:55,1.0,http://www.studentdoc.com
1726042,4777890,whitetail mining accident,2006-03-18 02:11:54,NaN,NaN
1668846,4517678,bookbinding washington dc,2006-05-15 12:20:52,1.0,http://www.superpages.com
2359298,7969898,zacky vengeance,2006-03-31 20:59:34,NaN,NaN
3333533,18226183,aa.com subscribe,2006-05-12 18:47:28,1.0,http://www.freefrequentflyermiles.com
574092,1465696,liz trotta's birth date,2006-05-08 06:02:04,NaN,NaN
3425012,20879890,fre-mont rideout hospital,2006-05-24 15:55:37,1.0,http://www.frhg.org


### 1.2. [10] Build a Trie

We want a suggest function to be **non-sensitive to stop words** because we don't want to upset the users if they confuses/omits prepositions. Consider *"public events in Innopolis"* vs *"public events at Innopolis"* or *"public events Innopolis"* - they all mean the same.

Build a Trie based on the dataset, **storing query statistics such as query _frequency_, urls and ranks in the nodes**. Some queries may have no associated urls, others may have multiple ranked urls. Think of the way to store this information.

Pass the asserts.

In [7]:
def build_aol_trie(aol_data, stops):
    trie = pygtrie.CharTrie()
    freqs = aol_data['Query'].value_counts()
    
    for _, row in tqdm(aol_data.iterrows(), total=len(aol_data)):
        query = row['Query']
        if pd.isna(query):
            continue
        _query = query[:]
        freq = freqs.get(query, None)
        queries = [_query[:]]
        _query = " ".join(w for w in _query.split() if w not in stops)

        urls = [row['ClickURL']]
        ranks = [row['ItemRank']]

        if _query in trie:
            freq, _urls, _ranks, _queries = trie[_query]
            urls += _urls
            ranks += _ranks
            queries += _queries
        trie[_query] = (freq, urls, ranks, queries)
    return trie

stops = set('a on at of to is from for and with using the in &'.split())
aol_trie = build_aol_trie(aol_data, stops)

100%|██████████| 3558411/3558411 [13:28<00:00, 4402.71it/s]


In [8]:
# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a property of list type. But you can do something different. 
    # bag += val.urls
    bag += val[1]
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

sample question surveys ~ (5, [nan, 'http://www.lg-employers.gov.uk', 'http://www.askemployees.com', 'http://www.custominsight.com', 'http://www.surveyconnect.com'], [nan, 1.0, 10.0, 4.0, 7.0], ['sample question surveys', 'sample question surveys', 'sample question surveys', 'sample question surveys', 'sample question surveys'])
sample questions immigration interview ~ (1, [nan], [nan], ['sample questions for immigration interview'])
sample questions interview ~ (1, ['http://www.quintcareers.com'], [1.0], ['sample questions for interview'])
sample questions family interview ~ (3, ['http://jobsearchtech.about.com', 'http://www.quintcareers.com', 'http://www.grandparents-day.com'], [3.0, 5.0, 2.0], ['sample questions for family interview', 'sample questions for family interview', 'sample questions for family interview'])
sample questions sociology race ethnicity ~ (1, [nan], [nan], ['sample questions sociology race and ethnicity'])
sample questions biology ~ (2, ['http://www.troy.k12.ny.

## 2. [15] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

Pass the asserts.

Question for analysis: What is the empirical threshold for minimal prefix for suggest?

In [9]:
stops = set('a on at of to is from for and with using the in &'.split())

In [10]:
def complete_user_query(query: str, trie_tree, top_k=5) -> list[str]:
    #TODO: suggest top_k options for a user query
    # sort results by frequency (!), 
    # suggest the QUERIES for first k ranked urls if available
    query = query[:].lower().strip()
    query_words = [word for word in query.split() if word not in stops]
    query = " ".join(query_words)
    
    items = list(trie_tree.iteritems(query))
    
    suggestions = sorted(items, key=lambda x: x[1][0], reverse=True)
    top_k = min(top_k, len(suggestions))
    top_suggestions = suggestions[:top_k]
    
    meta_data = {}
    for idx, suggestion in enumerate(top_suggestions):
        frequency, urls, ranks, queries = suggestion[1]
        most_common_query = Counter(queries).most_common(1)[0][0]
        
        meta_data[idx + 1] = {
            "Suggestion": most_common_query,
            "Query Count": frequency,
        }
        ranked_urls = [(url, rank) for url, rank in zip(urls, ranks) if url is not None and rank is not None]
        ranked_urls = sorted(ranked_urls, key=lambda x: x[1])
        if len(ranked_urls) > 0:
            meta_data[idx + 1]["URL"] = ranked_urls[0][0]
            meta_data[idx + 1]["URL rank"] = ranked_urls[0][1]
    
    pprint(meta_data)

    completions = [Counter(suggestion[1][3]).most_common(1)[0][0] for suggestion in top_suggestions]
    return completions

In [11]:
inp = "trie"
print("Query:", inp)
print("Results:")
res = complete_user_query(inp, aol_trie)
print(res)

#NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

assert "boys and girls club of conyers georgia" \
            in complete_user_query("boys girls club conyers", aol_trie, 10), "Should be here"

Query: trie
Results:
{1: {'Query Count': 5,
     'Suggestion': 'tried and true tattoo',
     'URL': 'http://www.triedntruetattoo.com',
     'URL rank': 1.0},
 2: {'Query Count': 3, 'Suggestion': 'triest', 'URL': nan, 'URL rank': nan},
 3: {'Query Count': 3,
     'Suggestion': 'triethanalomine',
     'URL': 'http://avalon.unomaha.edu',
     'URL rank': 1.0},
 4: {'Query Count': 2,
     'Suggestion': 'tried and failed',
     'URL': nan,
     'URL rank': nan},
 5: {'Query Count': 1,
     'Suggestion': "tried and truechildren's consignment sale",
     'URL': nan,
     'URL rank': nan}}
['tried and true tattoo', 'triest', 'triethanalomine', 'tried and failed', "tried and truechildren's consignment sale"]
{1: {'Query Count': 2,
     'Suggestion': 'boys and girls club of conyers georgia',
     'URL': 'http://www.bgcma.org',
     'URL rank': 1.0}}



Based on my experimental results, I have observed that a prefix of length 4-5 can provide a sufficient level of suggestion for a user query. In other words, I was able to obtain relevant suggestions with such prefixes during my experiments.

## 3. Measure suggest speed ##

### 3.1. [10] Full Trie test

Check how fast your search is working. Consider changing your code if it takes too long on average.

Sucess criterion:
- there is an average and variance for **multiple runs** of the given bucket.
- there is an average and variance for **multiple runs** of naive search in unindexed dataset.

In [12]:
inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

#TODO: measure average execution time and standard deviation (in milliseconds) per query and print it out
# Repeat this for index and for no index.

## 4. [10] Add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

You can use use [pyspellchecker](https://pypi.org/project/pyspellchecker/) `candidates()` call, or use any other spellchecker implementation.

In [13]:
def complete_user_query_with_spellchecker(query, trie, top_k=5) -> list[str]:
    #TODO: suggest top_k options for a user query
    # sort results by frequency (!!), 
    # suggest the QUERIES for first k ranked urls if available
    pass

In [ ]:
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]

for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, trie, 5) == \
            complete_user_query_with_spellchecker(q, trie, 5), "Assert {} and {} give different results".format(q, qc)